In [9]:
import pandas as pd
import numpy as np
import sklearn
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zero\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df1=pd.read_excel(r"datasets/extremism.xlsx")

df=pd.DataFrame()
df['id'] = df1['id']
df['text'] = df1['text']
df['class'] = df1['class']
df = df.dropna()

In [11]:
df

,id,text,class
0,1,Таким образом никто не внес большего вклад...,3.0
1,2,"Можно с уверенностью сказать, что все полезное...",3.0
2,3,Славяне должны на нас работать. Если они нам б...,3.0
3,4,"Русское государство росло, развивалось из свои...",3.0
4,5,Поставим власть под контроль народа,3.0
...,...,...,...
304,305,Социальное. Соответствующий подход предполагае...,0.0
305,306,Рост продуктивности человеческой деятельности ...,0.0
306,307,"Социальная дифференциация, в том числе по этни...",0.0
307,308,Демографический рост и расширение экономическо...,0.0


In [12]:
df.dropna(axis=0,inplace=True)

In [13]:
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
def comma2space(word):
    new_word = ''
    try:
        for char in word:
            if char in patterns:
                char = ' '
            new_word += char
    except TypeError:
        pass
    return new_word
    
def lemmatize(doc):
    doc = re.sub(patterns, '', doc)
    tokens = []
    for token in doc.split():
        if token:
            
            token = token.lower()
            token = token.strip()
            tokens.append(token)
    return tokens

def dropstopwords(docs):
    stopwords = nltk.corpus.stopwords.words('russian')
    sen =  []
    for doc in docs:
        if (doc not in stopwords):
            sen.append(doc)
    return sen

def dropcomma(doc):
    new_sen = ''
    for char in doc:
        if (char not in patterns):
            new_sen += char
    return new_sen
def tokenize(docs):
    docs = comma2space(docs)
    docs = lemmatize(docs)
    docs = dropstopwords(docs)
    return docs

In [14]:
data=df['text'].apply(tokenize)
data

0      [таким, образом, никто, внес, большего, вклада...
1      [уверенностью, сказать, полезное, хорошее, сов...
2      [славяне, должны, работать, нам, нужны, могут,...
3      [русское, государство, росло, развивалось, сво...
4                   [поставим, власть, контроль, народа]
                             ...                        
304    [социальное, соответствующий, подход, предпола...
305    [рост, продуктивности, человеческой, деятельно...
306    [социальная, дифференциация, числе, этническом...
307    [демографический, рост, расширение, экономичес...
308    [соответственно, политика, возникает, связи, у...
Name: text, Length: 309, dtype: object

In [47]:
import time
import multiprocessing
#WORD2VEC()
from gensim.models import Word2Vec
w2v_model = Word2Vec(
    min_count=10,
    window=10,
    size=300,
    negative=10,
    alpha=0.5,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1,workers=4)

#BUILD_VOCAB()
t = time.time()
w2v_model.build_vocab(data, progress_per=100)
print("Time to build vocab': {} mins".format(round((time.time() - t) / 60, 2)))

#TRAIN()
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time.time() - t) / 60, 2)))

Time to build vocab': 0.0 mins
Time to train the model: 0.0 mins


In [48]:
w2v_model.init_sims(replace=True)
w2v_model.wv.save_word2vec_format('hachi.bin', binary=True)

In [49]:
model1 = w2v_model.wv.load_word2vec_format(r'C:\Users\Zero\hachi.bin', binary=True)
model1.syn0

<ipython-input-49-f66df250e90e>:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  model1.syn0


array([[-0.04750724, -0.09294917,  0.04646128, ..., -0.02326778,
        -0.02748158,  0.04694638],
       [-0.16215153, -0.03675573,  0.03022764, ...,  0.01612221,
         0.01322101, -0.03014729],
       [-0.05710144, -0.07443115,  0.06365845, ...,  0.00222445,
         0.00118704,  0.08087655],
       ...,
       [-0.12255844, -0.10139921, -0.03399283, ..., -0.02904652,
         0.03170789, -0.04719374],
       [-0.07217082, -0.05173732,  0.02180016, ...,  0.03493413,
        -0.015422  , -0.08324368],
       [-0.04941457, -0.06038683,  0.01956714, ..., -0.01193344,
        -0.02067705,  0.00612094]], dtype=float32)

In [64]:
model1.most_similar(positive=["смерть","хачи"])

[('будут', 0.893851637840271),
 ('россии', 0.874309778213501),
 ('хачам', 0.8688006401062012),
 ('чурок', 0.8652452230453491),
 ('убивайте', 0.8409756422042847),
 ('умереть', 0.7862440943717957),
 ('самоубийства', 0.7742125988006592),
 ('знаю', 0.7616627216339111),
 ('это', 0.7607013583183289),
 ('думаю', 0.7601228356361389)]

In [65]:
import numpy as np
from scipy import spatial

index2word_set = set(model1.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

<ipython-input-65-37b38b84b612>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  index2word_set = set(model1.wv.index2word)


In [66]:
listok=['хач','убивай ',"негр"]
deist='всех'
score=[]
slovar={}
for i in range(len(listok)):
    s1_afv = avg_feature_vector(deist, model=model1, num_features=300, index2word_set=index2word_set)
    s2_afv = avg_feature_vector(listok[i], model=model1, num_features=300, index2word_set=index2word_set)
    sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
    score.append(sim)
    slovar.update({deist+' / '+listok[i]:sim})
sorted_tuples = sorted(slovar.items(), key=lambda item: item[1])

sorted_dict = {k: v for k, v in sorted_tuples}

print(sorted_dict)  # {1: 1, 3: 4, 2: 9}

{'всех / хач': nan, 'всех / убивай ': nan, 'всех / негр': nan}
